In [162]:
import pandas as pd
import sklearn
from sklearn import linear_model
from sklearn import preprocessing
import numpy as np
import logistic_regression as lr

In [2]:
expr = pd.read_csv("../input/expression.short.csv", index_col=(0,1))

In [3]:
g_df = pd.read_csv("../input/g_train_new.csv", index_col=(0,1))

In [8]:
tissue_groups = pd.read_csv("../tissue_groups/tissue_groups.physiological.txt", header=None, index_c

In [10]:
def processTissueGroups(tissue_groups_path):
    tissue_groups = {}
    f = open(tissue_groups_path)
    for l in f:
        w = l.strip().split(',')
        group = w[0]
        tissue_groups[group] = []
        for tissue in w[1:]: tissue_groups[group].append(tissue)
    return tissue_groups

In [144]:
tissue_groups = processTissueGroups("../tissue_groups/tissue_groups.physiological.txt")

In [145]:
tissue_categories = tissue_groups

In [5]:
g_df.head()

max_GC_10kb  max_CpG_10kb  max_priPhCons_10kb  \
subject_id gene_id                                                             
GTEX-N7MS  ENSG00000001561.6      1.015867      0.572087            0.556651   
           ENSG00000003056.3      1.156892      0.592920            0.566036   
           ENSG00000003402.15     1.451764      0.801254            0.553522   
           ENSG00000004534.10     1.323559      0.676254            0.579444   
           ENSG00000004660.10     1.323559      0.592920            0.571846   

                               max_mamPhCons_10kb  max_verPhCons_10kb  \
subject_id gene_id                                                      
GTEX-N7MS  ENSG00000001561.6             0.560247            0.578286   
           ENSG00000003056.3             0.563769            0.582093   
           ENSG00000003402.15            0.559807            0.577017   
           ENSG00000004534.10            0.584898            0.602396   
           ENSG00000004660.10            0.565090            0.583785   

                               max_priPhyloP_10kb  max_mamPhyloP_10kb  \
subject_id gene_id                                                      
GTEX-N7MS  ENSG00000001561.6             0.855699            0.799545   
           ENSG00000003056.3             0.903025            0.805725   
           ENSG00000003402.15            0.912318            0.797242   
           ENSG00000004534.10            0.953449            0.841063   
           ENSG00000004660.10            0.970400            0.875053   

                               max_verPhyloP_10kb  max_GerpN_10kb  \
subject_id gene_id                                                  
GTEX-N7MS  ENSG00000001561.6             0.672983        1.797529   
           ENSG00000003056.3             0.676492        1.769976   
           ENSG00000003402.15            0.668350        1.863979   
           ENSG00000004534.10            0.704711        1.350203   
           ENSG00000004660.10            0.733772        1.484887   

                               max_GerpS_10kb     ...       E113_EnhBiv  \
subject_id gene_id                                ...                     
GTEX-N7MS  ENSG00000001561.6         1.110027     ...                 0   
           ENSG00000003056.3         1.154362     ...                 0   
           ENSG00000003402.15        1.108385     ...                 0   
           ENSG00000004534.10        1.261642     ...                 0   
           ENSG00000004660.10        1.307126     ...                 0   

                               E116_general_promoter  E116_general_enhancer  \
subject_id gene_id                                                            
GTEX-N7MS  ENSG00000001561.6                       0                      0   
           ENSG00000003056.3                       0                      0   
           ENSG00000003402.15                      1                      0   
           ENSG00000004534.10                      1                      0   
           ENSG00000004660.10                      0                      1   

                               E116_TssA  E116_TssAFlnk  E116_EnhG  E116_Enh  \
subject_id gene_id                                                             
GTEX-N7MS  ENSG00000001561.6           0              0          0         0   
           ENSG00000003056.3           0              0          0         0   
           ENSG00000003402.15          1              0          0         0   
           ENSG00000004534.10          1              1          0         0   
           ENSG00000004660.10          0              0          0         1   

                               E116_TssBiv  E116_BivFlnk  E116_EnhBiv  
subject_id gene_id                                                     
GTEX-N7MS  ENSG00000001561.6             0             0            0  
           ENSG00000003056.3             0             0            0  
           ENSG00000003402.15    

In [261]:
cat = 'brain'
#tissues = tissue_categories[cat]
#tissues = brain_tissues_ganglia
tissues = brain_tissues_ganglia
#expr_cat = expr[tissue_categories[cat]]
expr_cat = expr[tissues]
#expr_cat = expr_cat.dropna(thresh = 2)
#if len(expr_cat.columns) <= 4:
#    expr_cat = expr_cat.dropna(thresh = 2)
#else:
#    expr_cat = expr_cat.dropna(thresh = 4)
#expr_cat = expr_cat.dropna()
expr_cat = expr_cat.dropna()
expr_cat["median"] = expr_cat.median(axis=1)
train = pd.concat([g_df, expr_cat["median"]], axis=1).dropna()
train["median_disc"] = sklearn.preprocessing.binarize(np.abs(train["median"].values).reshape(-1,1), threshold = 2.0)
G_train, G_test, y_train, y_test = sklearn.cross_validation.train_test_split(
train[g_df.columns].values, train["median_disc"].values, 
test_size=0.25, random_state=42)
beta = lr.sgd(G_train, y_train, np.ones(len(G_train[0])), beta_shared, 10000)
#beta = lr.sgd(G_train, y_train, np.ones(len(G_train[0])), np.zeros(len(G_train[0])), 100)

print("completed training.")
#l = sklearn.linear_model.LogisticRegression(penalty='l2', tol=.0001, fit_intercept=True)
#l.fit(G_train, y_train)
probas = lr.log_prob(G_test, beta)
print (cat, len(G_train), np.count_nonzero(y_train), sklearn.metrics.roc_auc_score(y_test, probas))
#print (cat, len(G_train), np.count_nonzero(y_train), sklearn.metrics.roc_auc_score(y_test, l.predict_proba(G_test)[:,1]))

Optimization terminated successfully.
         Current function value: 1651.951445
         Iterations: 127
         Function evaluations: 329
         Gradient evaluations: 329
completed training.
brain 13192 364 0.530124096056


In [213]:
brain_tissues_cortex

['Brain_Frontal_Cortex_BA9',
 'Brain_Cortex',
 'Brain_Anterior_cingulate_cortex_BA24']

In [177]:
beta_brain = beta

In [251]:
cat = 'brain'
#tissues = tissue_categories[cat]
tissues = brain_tissues_ganglia
#expr_cat = expr[tissue_categories[cat]]
expr_cat = expr[tissues]
#expr_cat = expr_cat.dropna(thresh = 2)
#if len(expr_cat.columns) <= 4:
#    expr_cat = expr_cat.dropna(thresh = 2)
#else:
#    expr_cat = expr_cat.dropna(thresh = 4)
#expr_cat = expr_cat.dropna()
#expr_cat = expr_cat.dropna(thresh = 5)
expr_cat["median"] = expr_cat.median(axis=1)
train = pd.concat([g_df, expr_cat["median"]], axis=1).dropna()
train["median_disc"] = sklearn.preprocessing.binarize(np.abs(train["median"].values).reshape(-1,1), threshold = 1.5)
G_train, G_test, y_train, y_test = sklearn.cross_validation.train_test_split(
train[g_df.columns].values, train["median_disc"].values, 
test_size=0.25, random_state=42)
beta = lr.sgd(G_train, y_train, np.ones(len(G_train[0])), beta_shared, .000001)
print("completed training.")
#l = sklearn.linear_model.LogisticRegression(penalty='l2', tol=.0001, fit_intercept=True)
#l.fit(G_train, y_train)
print (cat, len(G_train), np.count_nonzero(y_train), sklearn.metrics.roc_auc_score(y_test, lr.log_prob(G_test, beta)))
#print (cat, len(G_train), np.count_nonzero(y_train), sklearn.metrics.roc_auc_score(y_test, l.predict_proba(G_test)[:,1]))

/Users/farhan.damani/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Optimization terminated successfully.
         Current function value: 6933.700299
         Iterations: 412
         Function evaluations: 434
         Gradient evaluations: 434
completed training.
brain 22502 2188 0.518431389058


In [181]:
beta_shared = beta

In [169]:
np.ones(len(G_train[0])).shape

(362,)

In [125]:
expr[tissues].head()

Brain_Frontal_Cortex_BA9  Brain_Cortex  \
subject_id gene_id                                                      
GTEX-N7MS  ENSG00000001561.6                  -0.786175           NaN   
           ENSG00000003056.3                  -0.371790           NaN   
           ENSG00000003402.15                 -0.876486           NaN   
           ENSG00000004534.10                 -0.521206           NaN   
           ENSG00000004660.10                 -0.706056           NaN   

                               Brain_Anterior_cingulate_cortex_BA24  \
subject_id gene_id                                                    
GTEX-N7MS  ENSG00000001561.6                               0.811069   
           ENSG00000003056.3                              -1.355650   
           ENSG00000003402.15                             -1.132055   
           ENSG00000004534.10                              0.879670   
           ENSG00000004660.10                             -1.275275   

                               Brain_Nucleus_accumbens_basal_ganglia  \
subject_id gene_id                                                     
GTEX-N7MS  ENSG00000001561.6                               -0.198743   
           ENSG00000003056.3                                0.098514   
           ENSG00000003402.15                              -1.197502   
           ENSG00000004534.10                              -0.583644   
           ENSG00000004660.10                              -1.535246   

                               Brain_Putamen_basal_ganglia  \
subject_id gene_id                                           
GTEX-N7MS  ENSG00000001561.6                     -1.676483   
           ENSG00000003056.3                      0.744628   
           ENSG00000003402.15                     0.293015   
           ENSG00000004534.10                     0.144055   
           ENSG00000004660.10                     0.544196   

                               Brain_Caudate_basal_ganglia  
subject_id gene_id                                          
GTEX-N7MS  ENSG00000001561.6                      0.270070  
           ENSG00000003056.3                     -1.388229  
           ENSG00000003402.15                    -0.728798  
           ENSG00000004534.10                    -0.045774  
           ENSG00000004660.10                    -0.821366

In [94]:
tissues

['Brain_Frontal_Cortex_BA9',
 'Brain_Cortex',
 'Brain_Anterior_cingulate_cortex_BA24']

In [262]:
np.savetxt('./brain_cortex_labels_20.txt', y_test)

In [263]:
np.savetxt('./brain_cortex.scores_20.txt', probas)

In [241]:
len(y_test)

7501

In [106]:
tissue_categories[cat]

['Brain_Anterior_cingulate_cortex_BA24',
 'Brain_Caudate_basal_ganglia',
 'Brain_Cortex',
 'Brain_Frontal_Cortex_BA9',
 'Brain_Hippocampus']

In [146]:
brain_tissues = tissue_categories[cat]

In [147]:
brain_tissues

['Brain_Anterior_cingulate_cortex_BA24',
 'Brain_Caudate_basal_ganglia',
 'Brain_Cortex',
 'Brain_Frontal_Cortex_BA9',
 'Brain_Hippocampus',
 'Brain_Hypothalamus',
 'Brain_Nucleus_accumbens_basal_ganglia',
 'Brain_Putamen_basal_ganglia']

In [192]:
brain_tissues_cortex = ['Brain_Frontal_Cortex_BA9', 'Brain_Cortex', 'Brain_Anterior_cingulate_cortex_BA24']

In [116]:
tissues = ['Brain_Frontal_Cortex_BA9', 'Brain_Cortex', 'Brain_Anterior_cingulate_cortex_BA24', 'Brain_Nucleus_accumbens_basal_ganglia', 'Brain_Putamen_basal_ganglia', 'Brain_Caudate_basal_ganglia']

In [131]:
tissues_gang = ['Brain_Putamen_basal_ganglia', 'Brain_Caudate_basal_ganglia']

In [78]:
brain_tissues_ganglia = ['Brain_Nucleus_accumbens_basal_ganglia', 'Brain_Putamen_basal_ganglia', 'Brain_Caudate_basal_ganglia']

In [58]:
brain_tissues_cortex

['Brain_Frontal_Cortex_BA9',
 'Brain_Cortex',
 'Brain_Anterior_cingulate_cortex_BA24']